In [1]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification,AdamW, BertConfig, BertModel, BertPreTrainedModel
import pytorch_lightning as pl
#MODEL_NAME='bert-base-cased'
from torch import nn
import MeCab
from sklearn.preprocessing import LabelEncoder
batch=2
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tagger = MeCab.Tagger()
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [11]:
UserTweets = pd.read_csv("/home/is/shuntaro-o/dev/persons_move_analysis/data/20210_1month_per_hour/202210_1month_per_hour_ad_estimate.csv")
UserPlase = pd.read_csv('/home/is/shuntaro-o/dev/disaster_analysis_Twitter/data/TweetPlaces_Japan7Cities/FoursquareUserPlacesTokyo_20210419.txt', sep=',', lineterminator='\n', header=None,names = ['place_id', 'city_name', 'object', 'country', 'UserPlase_longitude', 'UserPlase_latitude', 'UserPlase_sub_longitude', 'UserPlase_sub_latitude'])

/tmp/ipykernel_58080/2688398077.py:1: DtypeWarning: Columns (7,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  UserTweets = pd.read_csv("/home/is/shuntaro-o/dev/persons_move_analysis/data/20210_1month_per_hour/202210_1month_per_hour_ad_estimate.csv")


In [12]:
UserTweets = UserTweets.iloc[:,0:-1]

In [13]:
UserPlase =UserPlase[~UserPlase.duplicated(subset='place_id')]
UserPlase = UserPlase.reset_index(drop=True)
UserTweets = UserTweets.reset_index(drop=True)
merged_Tweet_place = pd.merge(UserTweets , UserPlase, how='left', on = "place_id")
merged_Tweet_place = merged_Tweet_place.reset_index(drop=True)
merged_Tweet_place = merged_Tweet_place.dropna(subset=['UserPlase_latitude','UserPlase_longitude'])
def UTM_cal(lat,lon):
    p,a=divmod(lat*60,40)
    q,b=divmod(a,5)
    r,c=divmod(b*60,30)
    s,d=divmod(c,15)
    t,e=divmod(d,7.5)

    u=str(lon-100)[0:2]
    f=lon-int(u)-100
    v,g=divmod(f*60,7.5)
    w,h=divmod(g*60,45)
    x,i=divmod(h,22.5)
    y,j=divmod(i,11.25)

    m=(s*2)+(x+1)
    n=(t*2)+(y+1)
    p=int(p)
    u=int(u)
    q=int(q)
    v=int(v)
    r=int(r)
    w=int(w)
    m=int(m)
    n=int(n)

    ans=str(p)+str(u)+str(q)+str(v)
    ans=int(ans)
    return(ans)
code_list=[]
for index, r in merged_Tweet_place.iterrows():
    ans=UTM_cal(r.UserPlase_latitude,r.UserPlase_longitude)
    code_list.append(ans)
merged_Tweet_place = merged_Tweet_place.reset_index(drop=True)
code_list=pd.Series(code_list)
merged_Tweet_place=pd.concat([merged_Tweet_place,code_list],axis=1)
merged_Tweet_place=merged_Tweet_place.rename(columns={0: 'code'})

In [16]:
df = pd.read_csv("/home/is/shuntaro-o/dev/persons_move_analysis/data/202271month_per_hour_geotaged_adGeocode_Tokyo.csv")

In [17]:
y = "code"
L = LabelEncoder()
df[y] = L.fit_transform(df[y])
num_class = df[y].max() + 1  # オリジンの補正

In [18]:
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers import BertModel
class BertForSequenceClassifier_pl(pl.LightningModule):
        
    def __init__(self, model_name, lr, num_class):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_class)
        self.criterion = nn.CrossEntropyLoss()

        # BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        preds= self.classifier(output.pooler_output)
        
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        #print(f"tihi is {loss}")
        return loss, preds
    
       # trainのミニバッチに対して行う処理
    def training_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        self.log('train_loss', loss)
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # validation、testでもtrain_stepと同じ処理を行う
    def validation_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    def test_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # epoch終了時にvalidationのlossとaccuracyを記録
    def validation_epoch_end(self, outputs, mode="val"):
        # loss計算
        epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f"{mode}_loss", epoch_loss, logger=True)

        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"{mode}_accuracy", epoch_accuracy, logger=True)

        

    # testデータのlossとaccuracyを算出（validationの使いまわし）
    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, "test")

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [19]:
best_model_path = '/home/is/shuntaro-o/dev/persons_move_analysis/models/Tokyo_model_ver2.ckpt'

In [20]:
df_Tokyo = merged_Tweet_place

In [21]:
x = "text"
df_Tokyo[x] = df_Tokyo[x].astype(str)
df_Tokyo[x]=df_Tokyo[x].astype(str)
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[【】]', ' ')       # 【】の除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[（）()]', ' ')     # （）の除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
df_Tokyo[x]= df_Tokyo[x].str.replace(r'_', '')#underscodf_Tokyo[x].str 
df_Tokyo[x] = df_Tokyo[x].str.replace(r'https?:\/\/.*?[\r\n ]', '')  # URLの除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'http:\/\/.*?[\r\n ]', '')  # URLの除去
df_Tokyo[x] = df_Tokyo[x].str.replace(r'　', ' ')

/tmp/ipykernel_58080/2582238918.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[【】]', ' ')       # 【】の除去
/tmp/ipykernel_58080/2582238918.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[（）()]', ' ')     # （）の除去
/tmp/ipykernel_58080/2582238918.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
/tmp/ipykernel_58080/2582238918.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
/tmp/ipykernel_58080/2582238918.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_Tokyo[x] = df_Tokyo[x].str.replace(r'https?:\/\/.*?[\r\n ]', ''

In [23]:
sentences_text_test=df_Tokyo.text.values
list_estimate = []
model = BertForSequenceClassifier_pl.load_from_checkpoint(best_model_path)
bert=model.bert.cuda()
classifier=model.classifier.cuda()
for sentence in sentences_text_test:
    encoding = tokenizer(
    sentence,
    padding = 'longest',
    return_tensors='pt')
    encoding = { k: v.cuda() for k, v in encoding.items() }
    with torch.no_grad():
        output = bert(**encoding)
        ans=classifier(output.pooler_output)
        ans = ans.to('cpu').detach().numpy().copy()
        ans = ans.argmax()
        ans = L.inverse_transform([ans])[0]
        list_estimate.append(ans)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
list_estimate=pd.Series(list_estimate)
df_Tokyo = df_Tokyo.reset_index(drop=True)
df_Tokyo_adEstimate=pd.concat([df_Tokyo,list_estimate],axis=1)
df_Tokyo_adEstimate=df_Tokyo_adEstimate.rename(columns={0: 'Tokyo_code_estimate'})

In [25]:
df_Tokyo_adEstimate

,Unnamed: 0.1,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,place_id,author_id,...,author_location,city_name,object,country,UserPlase_longitude,UserPlase_latitude,UserPlase_sub_longitude,UserPlase_sub_latitude,code,Tokyo_code_estimate
0,0,0,それも良いけどロリデ行こうよ🏰👗🤴👸,2022-10-01_00:59:57,0.0,1.0,0.0,0,65b41c11aca77bdb,4843678447,...,ディズニーランド近辺,千葉 浦安市,city,日本,139.872407,35.616612,139.939631,35.672793,533936,533945
1,1,1,優良ドライバーさん🥇,2022-10-01_00:59:54,0.0,0.0,1.0,0,f1299e526e0cc34d,1312247388745023488,...,NaN,千葉 佐倉市,city,日本,140.126120,35.624589,140.301034,35.766545,534031,533934
2,2,2,ここ最近ずっとネガティブとポジティブが戦ってて、やっとポジティブが勝ちもうした！拍手！,2022-10-01_00:59:53,0.0,0.0,0.0,0,77173905596af4ab,449521349,...,NaN,東京 文京区,city,日本,139.717275,35.699474,139.772477,35.735929,533945,533945
3,3,3,有識者が教えてくれたらいいなぁw,2022-10-01_00:59:52,0.0,0.0,2.0,0,4c1c4868f96634ed,1060387314369683456,...,東京,東京 江東区,city,日本,139.771307,35.582057,139.849007,35.708066,533926,533934
4,4,4,ご無事で何よりです！おデート楽しんでください❤️‍🔥,2022-10-01_00:59:50,0.0,1.0,1.0,0,db215c78c59027e4,1573305402208686081,...,NaN,千葉 市川市,city,日本,139.885492,35.655568,139.976610,35.775796,533937,533935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69884,74427,95,今日の #夕暮れ はなんとも心動かされる色合いだった。鉄塔もいい感じに撮れた / today...,2022-10-31_23:57:58,0.0,0.0,2.0,0.0,1ea0fd1a31366833,7199952.0,...,横浜市,神奈 横浜市 都筑区,city,日本,139.543097,35.507956,139.616102,35.566753,533924,533914
69885,74428,96,お台場 Tokyoに写真を投稿しましたhttps://t.co/GJcY9s06SY,2022-10-31_23:57:57,0.0,0.0,0.0,0.0,594fa6c6bc5b5ba9,110322291.0,...,日本 東京,東京 港区,city,日本,139.708754,35.622973,139.782004,35.682605,533935,533935
69886,74429,97,おめでとうございます㊗️㊗️㊗️作品、必ず拝見します👀,2022-10-31_23:57:55,0.0,1.0,1.0,0.0,bcf7dca0cae1c2e6,1088408401.0,...,NaN,神奈 横浜市 南区,city,日本,139.570896,35.406284,139.641197,35.441318,533904,533944
69887,74430,98,これにつきまして、俺のプロレスデビューの時、感極まる人いただろうか…🤣笑いしか起きなかった様...,2022-10-31_23:57:55,0.0,0.0,1.0,0.0,a398e1f1025abe37,814713932.0,...,東京 北区,埼玉 春日部市,city,日本,139.707880,35.935390,139.832566,36.043504,533975,533945


In [27]:
df_Tokyo_adEstimate.to_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/202210_1month_per_hour_ad_estimate.csv')